In [24]:
# papermill parameters
output_dir = '../output/'
execution_date = '2020-03-28'
execution_time = '2020-03-28T00:00:00'

In [25]:
print('Date: %s Time: %s' % (execution_date, execution_time))

Date: 2020-03-28 Time: 2020-03-28T00:00:00


In [26]:
import pickle
import csv
from googleapiclient.discovery import build

spreadsheet_id = '1xw87zTCcM_Kgu0w9J6ecYdgcLAASHE7rRqluzPUgurk'
esp_cell_range = 'ESP AICS CLEAN!A1:R'

datasets_dir = "%s../datasets" % output_dir
credentials_dir = "%s../credentials" % output_dir

with open(credentials_dir + '/g_oauth_clt.pickle', 'rb') as token:
    creds = pickle.load(token)

# cache_discovery=False fixes superfluous import errors
gapi = build('sheets', 'v4', credentials=creds, cache_discovery=False)
gapi_sheets = gapi.spreadsheets()

def download_sheet(cell_range, dataset_file):
    result = gapi_sheets.values().get(spreadsheetId=spreadsheet_id, range=cell_range).execute()
    values = result.get('values', [])

    file = datasets_dir + dataset_file
    # If newline='' is not specified, newlines embedded inside quoted fields
    # will not be interpreted correctly, and on platforms that use \r\n
    # linendings on write an extra \r will be added. It should always be
    # safe to specify newline='', since the csv module does its own
    # (universal) newline handling
    with open(file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
        if not values:
            print('No data to process')
        else:
            for i, row in enumerate(values):
                writer.writerow(row)
    
    return values

In [27]:
import pandas as pd
esp_values = download_sheet(esp_cell_range, 'dswd_esp_aics.csv')

In [28]:
esp = pd.DataFrame(esp_values[1:], columns=esp_values[0])
esp = esp[esp['municity'] != '']
esp['adm2'] = esp['prov'].str.upper()
esp['adm2'] = esp['adm2'].str.replace(' (NOT A PROVINCE)', '', regex=False)
esp['adm3'] = esp['municity'].str.upper()
esp['adm3'] = esp['adm3'].str.replace('CITY OF ', '', regex=False)
esp['adm3'] = esp['adm3'].str.replace(' CITY', '', regex=False)
esp['adm3'] = esp['adm3'].str.replace(' \([A-Z\s\.\-\Ñ]+\)', '', regex=True) # Removes CAPITAL and (Alternate Names)
esp

,region,prov,municity,percent_cnt,percent_amt,target_cnt_4p,target_cnt_n4p,total_cnt_target,actual_cnt_4p,actual_cnt_n4p,total_cnt_actual,target_amt_4p,target_amt_n4p,total_amt_target,actual_amt_4p,actual_amt_n4p,total_amt_actual,actual_amt_lgu_n4p,adm2,adm3
104,IV-B,PALAWAN,ABORLAN,0.9885131856,0.9906526381,2356,3825,6181,2285,3825,6110,8599400,19125000,27724400,8340250,19125000,27465250,19125000,PALAWAN,ABORLAN
105,IV-B,OCCIDENTAL MINDORO,ABRA DE ILOG,0.998540856,0.9987485714,3400,2768,6168,3391,2768,6159,12410000,13840000,26250000,12377150,13840000,26217150,13840000,OCCIDENTAL MINDORO,ABRA DE ILOG
106,III,BATAAN,ABUCAY,0.9935108153,0.9935303578,840,5170,6010,834,5137,5971,4326000,33605000,37931000,4295100,33390500,37685600,33390500,BATAAN,ABUCAY
107,II,CAGAYAN,ABULUG,0.9970121088,0.9964194708,1009,5350,6359,990,5350,6340,4187350,29425000,33612350,4067000,29425000,33492000,29425000,CAGAYAN,ABULUG
108,VIII,LEYTE,ABUYOG,0.9977212007,0.9982008121,3554,9172,12726,3525,9172,12697,12972100,45860000,58832100,12866250,45860000,58726250,45860000,LEYTE,ABUYOG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732,BARMM,LANAO DEL SUR,WAO,0.9683794466,0.9739468269,2457,3362,5819,2273,3362,5635,8968050,16810000,25778050,8296450,16810000,25106450,16810000,LANAO DEL SUR,WAO
1733,VII,NEGROS ORIENTAL,ZAMBOANGUITA,0.9909509202,0.992686896,1189,5331,6520,1130,5331,6461,5528850,31986000,37514850,5254500,31986000,37240500,31986000,NEGROS ORIENTAL,ZAMBOANGUITA
1734,III,NUEVA ECIJA,ZARAGOZA,0.9954697835,0.9958697812,2117,8920,11037,2087,8900,10987,10902550,57980000,68882550,10748050,57850000,68598050,57850000,NUEVA ECIJA,ZARAGOZA
1735,VI,ILOILO,ZARRAGA,0.999614495,0.9992205415,973,4215,5188,971,4215,5186,4538400,25290000,29828400,4515150,25290000,29805150,25290000,ILOILO,ZARRAGA


In [29]:
adm3_csv = '{}/psgc_adm3.csv'.format(datasets_dir)
adm3 = pd.read_csv(adm3_csv)
adm3.columns = adm3.columns.str.lower()
adm3['adm2'] = adm3['adm2_en'].str.upper()
adm3['adm3'] = adm3['adm3_en'].str.upper()
adm3['adm3'] = adm3['adm3'].str.replace('CITY OF ', '', regex=False)
adm3['adm3'] = adm3['adm3'].str.replace(' CITY', '', regex=False)
adm3['adm3'] = adm3['adm3'].str.replace(' \([A-Z\s\.\-\Ñ]+\)', '', regex=True) # Removes CAPITAL and (Alternate Names)
adm3

,adm_id,adm1_pcode,adm1_en,adm2_pcode,adm2_en,adm3_pcode,adm3_en,latitude,longitude,area,perimeter,adm2,adm3
0,21523000,PH020000000,Region II,PH021500000,Cagayan,PH021523000,Santa Ana,18.393,122.221,4.347802e+08,198766.879,CAGAYAN,SANTA ANA
1,51711000,PH050000000,Region V,PH051700000,Camarines Sur,PH051711000,Caramoan,13.798,123.833,2.736764e+08,294694.792,CAMARINES SUR,CARAMOAN
2,97332000,PH090000000,Region IX,PH097300000,Zamboanga del Sur,PH097332000,Zamboanga City,7.156,122.145,1.497155e+09,521649.107,ZAMBOANGA DEL SUR,ZAMBOANGA
3,97338000,PH090000000,Region IX,PH097300000,Zamboanga del Sur,PH097338000,Pitogo,7.447,123.339,7.414699e+07,86993.113,ZAMBOANGA DEL SUR,PITOGO
4,98308000,PH090000000,Region IX,PH098300000,Zamboanga Sibugay,PH098308000,Malangas,7.620,123.008,1.210915e+08,113381.704,ZAMBOANGA SIBUGAY,MALANGAS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629,128002000,PH120000000,Region XII,PH128000000,Sarangani,PH128002000,Glan,5.785,125.342,7.086928e+08,166557.659,SARANGANI,GLAN
1630,97209000,PH090000000,Region IX,PH097200000,Zamboanga del Norte,PH097209000,Piñan,8.418,123.450,1.231303e+08,86076.222,ZAMBOANGA DEL NORTE,PIÑAN
1631,37703000,PH030000000,Region III,PH037700000,Aurora,PH037703000,Dilasag,16.400,122.097,4.954801e+08,121702.223,AURORA,DILASAG
1632,112324000,PH110000000,Region XI,PH112300000,Davao del Norte,PH112324000,San Isidro,7.749,125.706,1.634995e+08,81768.854,DAVAO DEL NORTE,SAN ISIDRO


In [31]:
dswd_adm2_csv = '{}/dswd_adm2_lookup.csv'.format(datasets_dir)
dswd_adm2 = pd.read_csv(dswd_adm2_csv)
dswd_adm2['adm2'] = dswd_adm2['lookup_key_adm2']
esp_adm2_adj = esp.merge(dswd_adm2, on=['adm2'], how='left')

esp_adm2_adj_y = esp_adm2_adj[esp_adm2_adj['lookup_adm2'].notnull()].copy()
esp_adm2_adj_n = esp_adm2_adj[esp_adm2_adj['lookup_adm2'].isnull()].copy()

esp_adm2_adj_y['adm2'] = esp_adm2_adj_y['lookup_adm2'].combine_first(esp_adm2_adj_y['adm2'])
esp_adm2_adj_y = esp_adm2_adj_y.drop(columns=['lookup_key_adm2', 'lookup_adm2'])
esp_adm2_adj_n = esp_adm2_adj_n.drop(columns=['lookup_key_adm2', 'lookup_adm2'])

esp_adm2_adj = pd.concat([esp_adm2_adj_y, esp_adm2_adj_n], ignore_index=True)

In [32]:
dswd_adm3_csv = '{}/dswd_adm3_lookup.csv'.format(datasets_dir)
dswd_adm3 = pd.read_csv(dswd_adm3_csv)
dswd_adm3['adm2'] = dswd_adm3['lookup_key_adm2']
dswd_adm3['adm3'] = dswd_adm3['lookup_key_adm3']
esp_adm3_adj = esp_adm2_adj.merge(dswd_adm3, on=['adm2', 'adm3'], how='left')

esp_adm3_adj_y = esp_adm3_adj[esp_adm3_adj['lookup_adm3'].notnull()].copy()
esp_adm3_adj_n = esp_adm3_adj[esp_adm3_adj['lookup_adm3'].isnull()].copy()

esp_adm3_adj_y['adm2'] = esp_adm3_adj_y['lookup_adm2'].combine_first(esp_adm3_adj_y['adm2'])
esp_adm3_adj_y['adm3'] = esp_adm3_adj_y['lookup_adm3'].combine_first(esp_adm3_adj_y['adm3'])
esp_adm3_adj_y = esp_adm3_adj_y.drop(columns=['lookup_key_adm2', 'lookup_key_adm3', 'lookup_adm2', 'lookup_adm3'])
esp_adm3_adj_n = esp_adm3_adj_n.drop(columns=['lookup_key_adm2', 'lookup_key_adm3', 'lookup_adm2', 'lookup_adm3'])

esp_adm3_adj = pd.concat([esp_adm3_adj_y, esp_adm3_adj_n], ignore_index=True)

In [33]:
merged = esp_adm3_adj.merge(adm3, on=['adm3','adm2'], how='left')
matched = merged[merged['adm_id'].notnull()]
missing = merged[merged['adm_id'].isnull()]
missing[['adm2', 'adm3']]

,adm2,adm3


In [50]:
matched#[matched['adm3'] == 'MANILA']['adm_id']

467    133900000
Name: adm_id, dtype: int64

In [47]:
dswd_esp_aics_geo_csv = '{}/dswd_esp_aics_geo.csv'.format(output_dir)
matched.to_csv(dswd_esp_aics_geo_csv, index=False)

In [48]:
matched.columns

Index(['region', 'prov', 'municity', 'percent_cnt', 'percent_amt',
       'target_cnt_4p', 'target_cnt_n4p', 'total_cnt_target', 'actual_cnt_4p',
       'actual_cnt_n4p', 'total_cnt_actual', 'target_amt_4p', 'target_amt_n4p',
       'total_amt_target', 'actual_amt_4p', 'actual_amt_n4p',
       'total_amt_actual', 'actual_amt_lgu_n4p', 'adm2', 'adm3', 'adm_id',
       'adm1_pcode', 'adm1_en', 'adm2_pcode', 'adm2_en', 'adm3_pcode',
       'adm3_en', 'latitude', 'longitude', 'area', 'perimeter'],
      dtype='object')

In [38]:
import geopandas
import numpy as np

esp_aics_sel = matched[['adm_id', 'region', 'prov', 'municity', 'percent_cnt', 'percent_amt',
       'target_cnt_4p', 'target_cnt_n4p', 'total_cnt_target', 'actual_cnt_4p',
       'actual_cnt_n4p', 'total_cnt_actual', 'target_amt_4p', 'target_amt_n4p',
       'total_amt_target', 'actual_amt_4p', 'actual_amt_n4p',
       'total_amt_actual', 'actual_amt_lgu_n4p']]

adm3_shp = geopandas.read_file('{}/MunicipalitiesManilaMergedMin.shp'.format(datasets_dir))
adm3_shp.columns = adm3_shp.columns.str.lower()
dswd_esp_aics_geo = adm3_shp.merge(esp_aics_sel, on=['adm_id'], how='left')
dswd_esp_aics_geo = dswd_esp_aics_geo.replace('', 'NaN')
convert_dict = {
    'percent_cnt': 'float64', 
    'percent_amt': 'float64',
    'target_cnt_4p': 'float64', 
    'target_cnt_n4p': 'float64',
    'total_cnt_target': 'float64', 
    'actual_cnt_4p': 'float64',
    'actual_cnt_n4p': 'float64', 
    'total_cnt_actual': 'float64', 
    'target_amt_4p': 'float64', 
    'target_amt_n4p': 'float64',
    'total_amt_target': 'float64', 
    'actual_amt_4p': 'float64', 
    'actual_amt_n4p': 'float64',
    'total_amt_actual': 'float64', 
    'actual_amt_lgu_n4p': 'float64'
}
dswd_esp_aics_geo = dswd_esp_aics_geo.astype(convert_dict)
#dswd_esp_aics_geo.dtypes


In [52]:
dswd_esp_aics_geo[dswd_esp_aics_geo['adm_id'] == 133900000]

,fid,cat,cat_,cat__1_1,cat__1,adm3_en,adm3_pcode,adm2_en,adm2_pcode,adm1_en,...,actual_cnt_4p,actual_cnt_n4p,total_cnt_actual,target_amt_4p,target_amt_n4p,total_amt_target,actual_amt_4p,actual_amt_n4p,total_amt_actual,actual_amt_lgu_n4p
5579,3422.0,1716,1837,5593,1232,City of Manila,PH133900000,"NCR, City of Manila, First District",PH133900000,National Capital Region,...,43823.0,185680.0,229503.0,298777850.0,1.485440e+09,1.784218e+09,291422950.0,1.485440e+09,1.776863e+09,1.485440e+09


In [54]:
dswd_esp_aics_geo_shp = '{}/dswd_esp_aics_geo.shp'.format(output_dir)
dswd_esp_aics_geo.to_file(dswd_esp_aics_geo_shp)